In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from torch.utils.tensorboard import SummaryWriter
from os import path as osp

/home/kelwa/anaconda3/envs/kaggle_torch/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 300  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 1  # number of epochs to train for (if early stopping is not triggered)
batch_size = 64
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder
pretrained_embeddings = True
fine_tune_embeddings = False
checkpoint = None  # path to checkpoint, None if none

In [5]:
DATA_NAME = 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop'

# local
DATA_JSON_PATH = 'data.json'
IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
# DATA_JSON_PATH = 'Image-Captioning/data.json'
# IMGS_PATH = 'flickr8k/images/'

In [6]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH); len(vocab)

100%|██████████| 40000/40000 [00:00<00:00, 388031.83it/s]


5089

### Pre-trained Embeddings

In [7]:
glove = pd.read_csv('glove.6B/glove.6B.300d.txt', sep=' ', quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()}

In [8]:
def create_embedding_matrix(vocab, embedding_dict, dimension):
    embedding_matrix = np.zeros((len(vocab), dimension))
    
    for word, index in vocab.stoi.items():
        if word in embedding_dict:
            embedding_matrix[index] = embedding_dict[word]
        else: 
            embedding_matrix[index] = np.random.uniform(-.1, .1, size=dimension)
    return embedding_matrix

In [9]:
embedding_matrix = create_embedding_matrix(vocab, glove_embedding, 300)

In [10]:
embedding_matrix.shape

(5089, 300)

In [15]:
with open('embeddings.pkl','wb') as f: pickle.dump(embedding_matrix, f)

with open('embeddings.pkl','rb') as f: embedding_matrix1 = pickle.load(f)

np.array_equal(embedding_matrix,embedding_matrix1) #sanity check

True

In [11]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
    'fine_tune_embeddings': fine_tune_embeddings,
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout,
    'embeddings_matrix': embedding_matrix
}

t_params

{'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_rmsprop',
 'imgs_path': 'flickr/Images/',
 'df_path': 'data.json',
 'vocab': <dataset.Vocabulary at 0x7f3d6dbd0350>,
 'epochs': 1,
 'batch_size': 64,
 'workers': 2,
 'decoder_lr': 0.0004,
 'encoder_lr': 0.0001,
 'fine_tune_encoder': False,
 'pretrained_embeddings': True,
 'fine_tune_embeddings': False}

In [12]:
fit(t_params=t_params, m_params=m_params)

Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [0][0/469]	Batch Time 6.728 (6.728)	Data Load Time 3.440 (3.440)	Loss 9.4371 (9.4371)	Top-5 Accuracy 0.000 (0.000)
Epoch: [0][100/469]	Batch Time 1.121 (1.090)	Data Load Time 0.000 (0.034)	Loss 5.1338 (5.7062)	Top-5 Accuracy 49.603 (43.075)
Epoch: [0][200/469]	Batch Time 1.105 (1.083)	Data Load Time 0.000 (0.017)	Loss 5.0455 (5.3644)	Top-5 Accuracy 50.611 (47.638)
Epoch: [0][300/469]	Batch Time 1.086 (1.072)	Data Load Time 0.000 (0.012)	Loss 4.6390 (5.1817)	Top-5 Accuracy 56.369 (50.053)
Epoch: [0][400/469]	Batch Time 1.001 (1.064)	Data Load Time 0.000 (0.009)	Loss 4.8202 (5.0634)	Top-5 Accuracy 56.081 (51.656)
-------------------- Validation -----------